In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
import os
import html


def load_model(model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2', save_path='C:/Users/tbula/PycharmProjects/jup_test/rugpt3'):
    try:
        if os.path.exists(save_path) and os.listdir(save_path):
            print("Загрузка модели с локального диска...")
            tokenizer = GPT2Tokenizer.from_pretrained(save_path)
            model = GPT2LMHeadModel.from_pretrained(save_path)
        else:
            print("Скачивание модели из интернета...")
            tokenizer = GPT2Tokenizer.from_pretrained(model_name)
            model = GPT2LMHeadModel.from_pretrained(model_name)
            os.makedirs(save_path, exist_ok=True)
            tokenizer.save_pretrained(save_path)
            model.save_pretrained(save_path)
    except Exception as e:
        print(f"Произошла ошибка при загрузке модели: {e}")
        raise e

    return model, tokenizer


def generate_text(model, tokenizer, prompt, device, max_new_tokens=200, temperature=0.7, top_k=50, top_p=0.95, num_beams=5, length_penalty=1.0):
    try:
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=max_new_tokens,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            num_beams=num_beams,
            length_penalty=length_penalty,
            pad_token_id=tokenizer.eos_token_id,
            top_k=top_k,
            top_p=top_p,
            temperature=temperature,
            do_sample=True,  # Включаем сэмплирование
            early_stopping=True
        )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        text = html.unescape(text)
    except Exception as e:
        print(f"Произошла ошибка при генерации текста: {e}")
        raise e
    return text


In [2]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2'
    save_path = 'C:/Users/tbula/PycharmProjects/jup_test/rugpt3'

    model, tokenizer = load_model(model_name, save_path)
    model.to(device)

    prompt = "привет"
    response = generate_text(model, tokenizer, prompt, device)
    print(response)

Using device: cuda
Загрузка модели с локального диска...
привет, как дела?

— Нормально, — ответил я. – А у тебя? Как дела с работой? Что-то я не видел тебя с тех пор, когда мы виделись в последний раз. Ты что-нибудь


In [3]:
import requests
from bs4 import BeautifulSoup

# Функция для извлечения ссылок со страницы
def extract_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True) if a['href'] and not a['href'].startswith('#')]
    return links

# Функция для извлечения текста с каждой страницы
def extract_text(url):
    response = requests.get(url)
    response.encoding = response.apparent_encoding  # Установим правильную кодировку
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([para.get_text() for para in paragraphs])
    return text

In [4]:
main_url = "https://helpdesk.bitrix24.ru/open/19070944/"
file_path = 'C:/Users/tbula/PycharmProjects/jup_test/bitrix_articles.txt'

if os.path.exists(file_path):
    print("Файл уже сохранен.")
else:
    article_links = extract_links(main_url)
    all_texts = []

    for link in article_links:
        if link.startswith('/'):
            link = "https://helpdesk.bitrix24.ru" + link
        elif not link.startswith('http'):
            continue  # Пропуск ссылок без схемы
        text = extract_text(link)
        all_texts.append(text)

    # Сохранение данных в файл
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as file:
        for text in all_texts:
            file.write(text + '\n')

    print("Данные собраны и сохранены.")

Данные собраны и сохранены.


In [5]:
from datasets import load_dataset
print(f"Using device: {device}")
dataset_path = 'C:/Users/tbula/PycharmProjects/jup_test/bitrix_articles.txt'
data = load_dataset('text', data_files=dataset_path)

Using device: cuda


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# Функция токенизации
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)  # Изменен max_length на 128

tokenized_data = data.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/2900 [00:00<?, ? examples/s]

In [7]:
# Группировка текстов для создания меток
def group_texts(examples):
    block_size = 128  # Изменен block_size на 128
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_data.map(group_texts, batched=True)
model.config.use_cache = False

Map:   0%|          | 0/2900 [00:00<?, ? examples/s]

In [8]:
import torch

print("PyTorch version: ", torch.__version__)
print("CUDA Available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())

if torch.cuda.is_available():
    print("GPU Name: ", torch.cuda.get_device_name(0))
else:
    print("No GPU available")

PyTorch version:  2.3.0+cu121
CUDA Available:  True
Number of GPUs:  1
GPU Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [9]:
import gc
# Очистка памяти перед тренировкой
# Очистка памяти перед тренировкой
gc.collect()
torch.cuda.empty_cache()

# Вывод сводки использования памяти до тренировки
print("Memory Summary before training:")
print(torch.cuda.memory_summary(device=device, abbreviated=False))

Memory Summary before training:
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1461 MiB |   1607 MiB |   6238 MiB |   4776 MiB |
|       from large pool |   1460 MiB |   1471 MiB |   1900 MiB |    439 MiB |
|       from small pool |      1 MiB |    146 MiB |   4337 MiB |   4336 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   1461 MiB |   1607 MiB |   6238 MiB |   4776 MiB |
|       from large pool |   1460

In [10]:
# Параметры тренировки
training_args = TrainingArguments(
    output_dir='C:/Users/tbula/PycharmProjects/jup_test/rugpt3_finetuned',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Уменьшенный размер батча
    gradient_accumulation_steps=1,  # Уменьшение аккумуляции градиентов
    save_steps=10_000,
    save_total_limit=2,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=1000,  # Частота оценки модели
    fp16=True,  # Использование смешанной точности, если поддерживается
    gradient_checkpointing=True,  # Включение градиентного контрольного пункта
)

torch.utils.checkpoint.use_reentrant = False

# Тренировка модели
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['train'],  # Можно использовать валидационный набор, если доступен
)

trainer.train()

model.save_pretrained('C:/Users/tbula/PycharmProjects/jup_test/rugpt3_finetuned')
tokenizer.save_pretrained('C:/Users/tbula/PycharmProjects/jup_test/rugpt3_finetuned')

print("Модель дообучена и сохранена.")

C:\Users\tbula\PycharmProjects\jup_test\venv\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


Модель дообучена и сохранена.


In [10]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    save_path = 'C:/Users/tbula/PycharmProjects/jup_test/rugpt3_finetuned'

    model, tokenizer = load_model(save_path)
    model.to(device)

    prompt = "как удалить базу знаний"
    response = generate_text(model, tokenizer, prompt, device)
    print(response)

Using device: cuda
Загрузка модели с локального диска...
как удалить базу знаний?
Панель управления\Программы и компоненты\Дополнительно\Установка и удаление программ.

Подскажите, пожалуйста, что это за порода собаки? (фото внутри)
двортерьер
Дворняга
Беспородная дворняжка, но очень красивая
беспородный щенок, скорее всего метис овчарки и лабрадора
Метис лабрикса и двортерьера. Очень милая собачка!
Лабрадор-ретривер, очень милая и добрая собака. Лабрадору не место в квартире, он должен жить в вольере или на выгуле, а не на улице. Дворняжки не могут уживаться с кошками и другими домашними животными, поэтому их надо выгуливать на поводке и в наморднике, чтобы они не кусались и не царапались. Если вы хотите завести собаку этой породы, то вам нужно будет найти хорошего заводчика, который сможет вырастить


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse

# URL сайта, который нужно парсить
base_url = 'https://helpdesk.bitrix24.ru/'

# Получение HTML-контента
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Функция для проверки валидности URL
def is_valid_url(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

# Функция для получения текста из всех ссылок на странице
def get_text_from_links(soup, base_url):
    links = soup.find_all('a', href=True)
    data = []
    for link in links:
        link_text = link.get_text(strip=True)
        link_url = urljoin(base_url, link['href'])
        if not is_valid_url(link_url):
            continue
        try:
            link_response = requests.get(link_url)
            link_soup = BeautifulSoup(link_response.content, 'html.parser')
            page_text = link_soup.get_text(separator=' ', strip=True)
            data.append({'link_text': link_text, 'link_url': link_url, 'page_text': page_text})
        except Exception as e:
            print(f"Не удалось получить данные с {link_url}: {e}")
    return data

# Получение текста из всех ссылок на главной странице
data = get_text_from_links(soup, base_url)

# Создание DataFrame и сохранение в CSV файл
df = pd.DataFrame(data)
df.to_csv('bitrix24_helpdesk_data.csv', index=False)

print('Датасет успешно сохранён в файл bitrix24_helpdesk_data.csv')

Датасет успешно сохранён в файл bitrix24_helpdesk_data.csv
